# Graph Attention Network (GAT) with DRFP Integration

**Problem**: CV-LB intercept (0.0525) > Target (0.0347). All tabular approaches fall on the same CV-LB line.

**Solution**: Use Graph Attention Networks (GAT) which learn which atoms/bonds matter for each prediction.

**Key insight from benchmark**: The benchmark paper achieved MSE 0.0039 using GAT + DRFP.

**This notebook has EXACTLY 3 submission cells at the end.**

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

# PyTorch Geometric imports
from torch_geometric.data import Data, Batch
from torch_geometric.nn import GATConv, global_mean_pool

# RDKit imports
from rdkit import Chem

np.random.seed(42)
torch.manual_seed(42)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
torch.set_default_dtype(torch.float32)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

Using device: cuda


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load SMILES lookup and DRFP features
SMILES_DF = pd.read_csv(f'{DATA_PATH}/smiles_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'SMILES lookup: {len(SMILES_DF)} solvents')
print(f'DRFP filtered: {DRFP_FILTERED.shape}')

SMILES lookup: 26 solvents
DRFP filtered: (24, 122)


In [4]:
# SMILES to molecular graph conversion with rich atom features
def smiles_to_graph(smiles):
    """Convert SMILES to PyTorch Geometric Data object with rich atom features."""
    if '.' in smiles:
        smiles = smiles.split('.')[0]
    
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError(f"Could not parse SMILES: {smiles}")
    
    # Rich atom features (9 features per atom)
    atom_features = []
    for atom in mol.GetAtoms():
        features = [
            atom.GetAtomicNum(),
            atom.GetDegree(),
            atom.GetFormalCharge(),
            int(atom.GetHybridization()),
            int(atom.GetIsAromatic()),
            atom.GetTotalNumHs(),
            atom.GetNumRadicalElectrons(),
            int(atom.IsInRing()),
            atom.GetMass() / 100.0,  # Normalized mass
        ]
        atom_features.append(features)
    
    x = torch.tensor(atom_features, dtype=torch.float)
    
    # Edge index from bonds
    edge_index = []
    for bond in mol.GetBonds():
        i = bond.GetBeginAtomIdx()
        j = bond.GetEndAtomIdx()
        edge_index.extend([[i, j], [j, i]])
    
    if len(edge_index) == 0:
        edge_index = [[0, 0]]
    
    edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
    
    return Data(x=x, edge_index=edge_index)

# Pre-compute all graphs
SOLVENT_GRAPHS = {}
for solvent_name in SMILES_DF.index:
    smiles = SMILES_DF.loc[solvent_name, 'solvent smiles']
    SOLVENT_GRAPHS[solvent_name] = smiles_to_graph(smiles)

print(f'Pre-computed {len(SOLVENT_GRAPHS)} solvent graphs')
print(f'Node feature dimension: {SOLVENT_GRAPHS[list(SOLVENT_GRAPHS.keys())[0]].x.shape[1]}')

Pre-computed 26 solvent graphs
Node feature dimension: 9


In [5]:
# Graph Attention Network Model
class GATModel(nn.Module):
    """Graph Attention Network for molecular property prediction."""
    
    def __init__(self, node_dim=9, hidden_dim=64, heads=4, drfp_dim=122, kinetics_dim=5):
        super().__init__()
        # GAT layers with multi-head attention
        self.conv1 = GATConv(node_dim, hidden_dim, heads=heads, dropout=0.2)
        self.conv2 = GATConv(hidden_dim * heads, hidden_dim, heads=1, dropout=0.2)
        
        # Combine graph embedding with DRFP and kinetics
        combined_dim = hidden_dim + drfp_dim + kinetics_dim
        
        self.predictor = nn.Sequential(
            nn.Linear(combined_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 3),
            nn.Sigmoid()
        )
    
    def forward(self, graph_batch, drfp, kinetics):
        x, edge_index, batch = graph_batch.x, graph_batch.edge_index, graph_batch.batch
        
        # GAT layers
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, edge_index)
        
        # Global mean pooling for graph-level representation
        x = global_mean_pool(x, batch)
        
        # Combine with DRFP and kinetics
        combined = torch.cat([x, drfp, kinetics], dim=-1)
        
        return self.predictor(combined)

print('GATModel defined')

GATModel defined


In [6]:
# GAT Model for mixed solvents
class GATModelMixed(nn.Module):
    """Graph Attention Network for mixed solvent prediction."""
    
    def __init__(self, node_dim=9, hidden_dim=64, heads=4, drfp_dim=122, kinetics_dim=5):
        super().__init__()
        # Shared GAT layers
        self.conv1 = GATConv(node_dim, hidden_dim, heads=heads, dropout=0.2)
        self.conv2 = GATConv(hidden_dim * heads, hidden_dim, heads=1, dropout=0.2)
        
        # Combine two graph embeddings + DRFP + kinetics + percentage
        combined_dim = hidden_dim * 2 + drfp_dim * 2 + kinetics_dim + 1
        
        self.predictor = nn.Sequential(
            nn.Linear(combined_dim, 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 3),
            nn.Sigmoid()
        )
    
    def _encode_graph(self, graph_batch):
        x, edge_index, batch = graph_batch.x, graph_batch.edge_index, graph_batch.batch
        x = F.relu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.2, training=self.training)
        x = self.conv2(x, edge_index)
        return global_mean_pool(x, batch)
    
    def forward(self, graph_batch_a, graph_batch_b, drfp_a, drfp_b, kinetics, pct):
        # Encode both graphs
        emb_a = self._encode_graph(graph_batch_a)
        emb_b = self._encode_graph(graph_batch_b)
        
        # Combine all features
        combined = torch.cat([emb_a, emb_b, drfp_a, drfp_b, kinetics, pct.unsqueeze(-1)], dim=-1)
        
        return self.predictor(combined)

print('GATModelMixed defined')

GATModelMixed defined


In [7]:
# GAT Wrapper class
class GATModelWrapper:
    """Wrapper for GAT model with training and prediction."""
    
    def __init__(self, data='single', hidden_dim=64, heads=4, n_models=3):
        self.data_type = data
        self.hidden_dim = hidden_dim
        self.heads = heads
        self.n_models = n_models
        self.models = []
        self.scaler = StandardScaler()
        
    def _get_kinetics(self, X):
        """Extract kinetics features."""
        time_m = X["Residence Time"].values.reshape(-1, 1)
        temp_c = X["Temperature"].values.reshape(-1, 1)
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        return np.hstack([time_m, temp_c, inv_temp, log_time, interaction]).astype(np.float32)
    
    def _get_graphs(self, solvent_names):
        """Get pre-computed graphs for solvents."""
        return [SOLVENT_GRAPHS[name].clone() for name in solvent_names]
    
    def _get_drfp(self, solvent_names):
        """Get DRFP features for solvents."""
        return DRFP_FILTERED.loc[solvent_names].values.astype(np.float32)
    
    def train_model(self, X_train, Y_train, epochs=150):
        Y_np = Y_train.values if hasattr(Y_train, 'values') else Y_train
        
        # Get features
        kinetics = self._get_kinetics(X_train)
        kinetics_scaled = self.scaler.fit_transform(kinetics)
        kinetics_tensor = torch.tensor(kinetics_scaled, dtype=torch.float32).to(device)
        Y_tensor = torch.tensor(Y_np, dtype=torch.float32).to(device)
        
        if self.data_type == 'single':
            graphs = self._get_graphs(X_train["SOLVENT NAME"].values)
            drfp = self._get_drfp(X_train["SOLVENT NAME"].values)
            drfp_tensor = torch.tensor(drfp, dtype=torch.float32).to(device)
        else:
            graphs_a = self._get_graphs(X_train["SOLVENT A NAME"].values)
            graphs_b = self._get_graphs(X_train["SOLVENT B NAME"].values)
            drfp_a = self._get_drfp(X_train["SOLVENT A NAME"].values)
            drfp_b = self._get_drfp(X_train["SOLVENT B NAME"].values)
            drfp_a_tensor = torch.tensor(drfp_a, dtype=torch.float32).to(device)
            drfp_b_tensor = torch.tensor(drfp_b, dtype=torch.float32).to(device)
            pct = torch.tensor(X_train["SolventB%"].values.astype(np.float32), dtype=torch.float32).to(device)
        
        # Train multiple models
        self.models = []
        drfp_dim = DRFP_FILTERED.shape[1]
        
        for seed in range(self.n_models):
            torch.manual_seed(42 + seed)
            
            if self.data_type == 'single':
                model = GATModel(hidden_dim=self.hidden_dim, heads=self.heads, drfp_dim=drfp_dim).to(device)
            else:
                model = GATModelMixed(hidden_dim=self.hidden_dim, heads=self.heads, drfp_dim=drfp_dim).to(device)
            
            optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-5)
            criterion = nn.HuberLoss()
            
            model.train()
            batch_size = 32
            n_samples = len(Y_np)
            
            for epoch in range(epochs):
                indices = torch.randperm(n_samples)
                for start in range(0, n_samples, batch_size):
                    end = min(start + batch_size, n_samples)
                    batch_idx = indices[start:end]
                    
                    batch_kinetics = kinetics_tensor[batch_idx]
                    batch_y = Y_tensor[batch_idx]
                    
                    if self.data_type == 'single':
                        batch_graphs = [graphs[i] for i in batch_idx]
                        batch_graph = Batch.from_data_list(batch_graphs).to(device)
                        batch_drfp = drfp_tensor[batch_idx]
                        pred = model(batch_graph, batch_drfp, batch_kinetics)
                    else:
                        batch_graphs_a = [graphs_a[i] for i in batch_idx]
                        batch_graphs_b = [graphs_b[i] for i in batch_idx]
                        batch_graph_a = Batch.from_data_list(batch_graphs_a).to(device)
                        batch_graph_b = Batch.from_data_list(batch_graphs_b).to(device)
                        batch_drfp_a = drfp_a_tensor[batch_idx]
                        batch_drfp_b = drfp_b_tensor[batch_idx]
                        batch_pct = pct[batch_idx]
                        pred = model(batch_graph_a, batch_graph_b, batch_drfp_a, batch_drfp_b, batch_kinetics, batch_pct)
                    
                    optimizer.zero_grad()
                    loss = criterion(pred, batch_y)
                    loss.backward()
                    optimizer.step()
            
            model.eval()
            self.models.append(model)
    
    def predict(self, X_test):
        kinetics = self._get_kinetics(X_test)
        kinetics_scaled = self.scaler.transform(kinetics)
        kinetics_tensor = torch.tensor(kinetics_scaled, dtype=torch.float32).to(device)
        
        if self.data_type == 'single':
            graphs = self._get_graphs(X_test["SOLVENT NAME"].values)
            graph_batch = Batch.from_data_list(graphs).to(device)
            drfp = self._get_drfp(X_test["SOLVENT NAME"].values)
            drfp_tensor = torch.tensor(drfp, dtype=torch.float32).to(device)
        else:
            graphs_a = self._get_graphs(X_test["SOLVENT A NAME"].values)
            graphs_b = self._get_graphs(X_test["SOLVENT B NAME"].values)
            graph_batch_a = Batch.from_data_list(graphs_a).to(device)
            graph_batch_b = Batch.from_data_list(graphs_b).to(device)
            drfp_a = self._get_drfp(X_test["SOLVENT A NAME"].values)
            drfp_b = self._get_drfp(X_test["SOLVENT B NAME"].values)
            drfp_a_tensor = torch.tensor(drfp_a, dtype=torch.float32).to(device)
            drfp_b_tensor = torch.tensor(drfp_b, dtype=torch.float32).to(device)
            pct = torch.tensor(X_test["SolventB%"].values.astype(np.float32), dtype=torch.float32).to(device)
        
        preds = []
        with torch.no_grad():
            for model in self.models:
                if self.data_type == 'single':
                    pred = model(graph_batch, drfp_tensor, kinetics_tensor)
                else:
                    pred = model(graph_batch_a, graph_batch_b, drfp_a_tensor, drfp_b_tensor, kinetics_tensor, pct)
                preds.append(pred.cpu())
        
        return torch.clamp(torch.stack(preds).mean(dim=0), 0, 1)

print('GATModelWrapper defined')

GATModelWrapper defined


In [8]:
# Quick test on single fold
X_single, Y_single = load_data("single_solvent")
test_solvent = sorted(X_single["SOLVENT NAME"].unique())[0]
mask = X_single["SOLVENT NAME"] != test_solvent

print(f"Test solvent: {test_solvent}")
print(f"Training samples: {mask.sum()}, Test samples: {(~mask).sum()}")

model = GATModelWrapper(data='single', n_models=2)
model.train_model(X_single[mask], Y_single[mask], epochs=100)
preds = model.predict(X_single[~mask])

actuals = Y_single[~mask].values
mse = np.mean((actuals - preds.numpy()) ** 2)
print(f'Test fold MSE: {mse:.6f}')

Test solvent: 1,1,1,3,3,3-Hexafluoropropan-2-ol
Training samples: 619, Test samples: 37


Test fold MSE: 0.074600


In [9]:
# Run full CV for single solvent
print('Running CV for single solvent data...')

X, Y = load_data("single_solvent")
split_generator = generate_leave_one_out_splits(X, Y)
all_preds = []
all_actuals = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = GATModelWrapper(data='single', n_models=3)
    model.train_model(train_X, train_Y, epochs=150)
    
    predictions = model.predict(test_X)
    predictions_np = predictions.detach().cpu().numpy()
    
    all_preds.append(predictions_np)
    all_actuals.append(test_Y.values)

all_preds = np.vstack(all_preds)
all_actuals = np.vstack(all_actuals)
mse_single = np.mean((all_preds - all_actuals) ** 2)
print(f'\nSingle Solvent MSE: {mse_single:.6f}')

Running CV for single solvent data...


0it [00:00, ?it/s]

1it [00:33, 33.25s/it]

2it [01:06, 33.40s/it]

3it [01:38, 32.68s/it]

...

22it [12:20, 34.25s/it]

23it [12:54, 34.05s/it]

24it [13:28, 33.95s/it]

24it [13:28, 33.67s/it]


Single Solvent MSE: 0.016634


In [ ]:
# Run full CV for full data
print('Running CV for full data...')

X, Y = load_data("full")
split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_preds_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split
    
    model = GATModelWrapper(data='full', n_models=3)
    model.train_model(train_X, train_Y, epochs=150)
    
    predictions = model.predict(test_X)
    predictions_np = predictions.detach().cpu().numpy()
    
    all_preds_full.append(predictions_np)
    all_actuals_full.append(test_Y.values)

all_preds_full = np.vstack(all_preds_full)
all_actuals_full = np.vstack(all_actuals_full)
mse_full = np.mean((all_preds_full - all_actuals_full) ** 2)
print(f'\nFull Data MSE: {mse_full:.6f}')

# Overall MSE
n_single = len(all_actuals)
n_full = len(all_actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)
print(f'\nOverall MSE: {overall_mse:.6f}')
print(f'Baseline (exp_030 GP+MLP+LGBM): CV 0.008298')

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = GATModelWrapper(data='single', n_models=3)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = GATModelWrapper(data='full', n_models=3)  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [ ]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################